In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import os
import gmaps
import seaborn as sns
import re

from statsmodels.compat import lzip
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Import API key from a file that is ignored by Git (.gitignore file) so the key isn't exposed to the public
#from config import gkey

# Configure gmaps
#gmaps.configure(api_key=gkey)

## Store County Health Rankings Excel file results into DataFrame

* Load the excel file imported from https://www.countyhealthrankings.org/app/texas/2019/measure/outcomes/144/description?sort=desc-2

In [ ]:
path = "data/2019 County Health Rankings Texas Data - v1_0.xls"

# Get the available sheets in the excel file and put into a list
sheets = pd.ExcelFile(path).sheet_names
print(sheets)

## Read in the Ranked Measure Data sheet eliminating columns and extra headers
* Make a list of available columns to decide what we don't want
* Formulate a regex expression to match unwanted columns
* Create a list of only wanted columns to be used for the usecols argument
* Set county to be the index
* Eliminate any row without a county name

In [ ]:
# Get the 'Ranked Measure Data' sheet using the sheets list above and rejecting the top header row
rmd = pd.read_excel(path, header=[1], sheet_name=sheets[3])

# View the columns and decide what we don't want
col1 = [col for col in rmd.columns] 
col1.sort()
col1

In [ ]:
# Create a regular expression to match unwanted columns
regex = '^Z|95|State|Unreliable'

# Use a list comprehension to make a list of columns that don't match the regex (a failed regex match returns 'None')
cols = [col for col in rmd.columns if re.match(regex,col) is None]

# Create a dataframe of the desired columns and set 'County' to the index
rmd_df = pd.read_excel(path, header=[1], sheet_name=sheets[3],usecols=cols)

# Drop any row that has NaN as County value
rmd_df = rmd_df[pd.notnull(rmd_df['County'])]
print(f'shape of rmd_df: {rmd_df.shape}')
rmd_df.head(1)

## Read in the Additional Measure Data sheet eliminating columns and extra headers
* Make a list of available columns to decide what we don't want
* Formulate a regex expression to match unwanted columns
* Create a list of only wanted columns to be used for the usecols argument
* Set county to be the index
* Eliminate any row without a county name

In [ ]:
# Get the 'Ranked Measure Data' sheet using the sheets list above and rejecting the top header row
amd = pd.read_excel(path, header=[1], sheet_name=sheets[4])

# View the columns and decide what we don't want
[print(col) for col in amd.columns] 

In [ ]:
# Create a regular expression to match unwanted columns
regex = '^Z|95|State|Unreliable'

# Use a list comprehension to make a list of columns that don't match the regex (a failed regex match returns 'None')
cols = [col for col in amd.columns if re.match(regex,col) is None]

# Create a dataframe of the desired columns and set 'County' to the index
amd_df = pd.read_excel(path, header=[1], sheet_name=sheets[4],usecols=cols)

# Drop any row that has Nan as County value
amd_df = amd_df[pd.notnull(amd_df['County'])]
print(f'shape of rmd_df: {amd_df.shape}')
amd_df.head(1)

In [ ]:
# Merge the rmd and amd dataframes using inner join on County column (index)
merged_df = pd.merge(rmd_df,amd_df, how='inner', on='County')
print(f'shape of merged_df: {merged_df.shape}')
#merged_df.head(1)

In [ ]:
# View the columns that end in '_x' or '_y'
regex2 = '.*_x$|.*_y$'
[print(col) for col in merged_df.columns if re.match(regex2,col) is not None] 

In [ ]:
# Drop the duplicate _y columns
regex3 = '.*_y$'
merged_df.drop([col for col in merged_df.columns if re.match(regex3,col) is not None],axis=1, inplace=True)
print(merged_df.shape)

In [ ]:
# Rename the columns to eliminate the '_x' leftover from the join
merged_df.rename(columns = {'FIPS_x':'FIPS',\
                                        '# Uninsured_x':'# Uninsured',\
                                        '% Uninsured_x':'% Uninsured',\
                                        'Population_x':'Population'},\
                                         inplace=True)
#merged_df.columns
# View the columns that end in '_x' or '_y'
regex2 = '.*_x$|.*_y$'
[print(col) for col in merged_df.columns if re.match(regex2,col) is not None] 

## Store NCHS Urban Rural Classification System Excel file results into DataFrame

* Load the excel file imported from https://www.cdc.gov/nchs/data_access/urban_rural.htm#Data_Files_and_Documentation
* No description or label was given in the 'NCHS URCS' - only numbers 1-6. Reading the file documentation gave this information, so an excel sheet was created and a VLOOKUP mapped the names and descriptions to the classification numbers. The result excel file will be imported and joined to the merged_df

In [ ]:
path2 = 'data/NCHS Urban Rural Classification System.xlsx'

# Get the available sheets in the excel file and put into a list
sheets2 = pd.ExcelFile(path2).sheet_names
print(sheets2)

In [ ]:
# Get the 'URCS' sheet using the sheets list above and rejecting the top header row
cs = pd.read_excel(path2)
cs.columns
cs.head(1)


In [ ]:
# Get the 'URCS' sheet using the columns list above
urcs = pd.read_excel(path2,usecols=['FIPS code','URCS','URCS Name','URCS description'],index_col=0)
print(f'shape of urcs: {urcs.shape}')


In [ ]:
# Drop any row that has Nan as an index value
urcs = urcs.loc[urcs.index.dropna()]
print(f'shape of urcs: {urcs.shape}')

In [ ]:
# Check that all FIPS in the merged_df are in the urcs.index (urcs index has ALL counties in USA not just Texas)
print(f"merged_df FIPS:{[mips for mips in merged_df['FIPS'] if mips not in urcs.index]} not found in urcs.index")
#print(f'urcs county:{[county for county in urcs.index if county not in merged_df["FIPS"]]} not found in merged_df.index')

In [ ]:
# Rename the index 'FIPS code' to FIPS
urcs.rename_axis('FIPS', axis=0, inplace=True)
urcs.head()

# Merge the NCHS URCS data with the merged_df

In [ ]:
all_df = pd.merge(urcs,merged_df, how='inner', on='FIPS')
#print(all_df.shape)
all_df.head(1)

In [ ]:
multi_df = all_df.set_index(['URCS','URCS Name','County'])
multi_df = multi_df.sort_index()

print(multi_df.shape)
multi_df.head(50)

In [ ]:
#idx = pd.IndexSlice
#rural_df = final_merged_df.loc[idx['Rural',:],:]
rural_df = multi_df.loc['Rural']
print(rural_df.shape)

In [ ]:
#urban_df = final_merged_df.loc[idx['Urban',:],:]
urban_df = multi_df.loc['Urban']
print(urban_df.shape)

In [ ]:
urban_df.head(3)

In [ ]:
corr1 = multi_df.corr()

In [ ]:
all_df.to_csv('all_df.csv')

In [ ]:
corr1.to_csv('Corr1.csv')

In [ ]:
multi_df.to_csv('multi_df.csv')

In [ ]:
corr1_spear = multi_df.corr(method='spearman')

In [ ]:
corr1_spear.to_csv('Corr1_spear.csv')

## Function to easily do a panel of scatterplots easily

In [ ]:
def makePanelScatterplot(data_df, x_data,y_data,color_col,split_col):
    x_label = x_data
    y_label = y_data
    ylim_min = data_df[y_data].min()*1.05
    ylim_max = data_df[y_data].max()*1.05
    xlim_min = data_df[x_data].min()*1.05
    xlim_max = data_df[x_data].max()*1.05

    g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col,col_wrap=3)

        
    g = (g.set_axis_labels(x_label, y_label)
          .set(ylim=(ylim_min, ylim_max),xlim=(xlim_min, xlim_max))
          .fig.subplots_adjust(wspace=.02))

## Function to call OLS Regression Statsmodel from 2 columns of dataframe

In [ ]:
def olsRegressionAnalysis (df,df_name,dep_col,ind_col):
    stat_dep_col = dep_col.replace(' ','_').replace('-','_')
    stat_ind_col = ind_col.replace(' ','_').replace('-','_')
    stat_col_list = [stat_dep_col,stat_ind_col]
    
    col_list = [dep_col,ind_col]

    col_dict = {col_list[i]: stat_col_list[i] for i in range(len(col_list))} 
    #print(col_dict)
    stat_df = df[col_list].dropna()
    stat_df.rename(columns=col_dict,inplace=True)
    print(f'\u001b[34m{dep_col}\u001b[0m fitted against \u001b[34m{ind_col}\u001b[0m using \x1b[31m{df_name}\x1b[0m dataframe:\n')
    print(f'We have {stat_df.shape[0]} rows left after dropping Null values\n')
    model_string = stat_dep_col + " ~ " + stat_ind_col
    all_model = ols(model_string, data=stat_df).fit()
    print(all_model.summary())

# Age adjusted Mortality (Deaths/100k) for Everyone

In [ ]:
makePanelScatterplot(all_df,"Age-Adjusted Mortality","Household Income","URCS","URCS")

In [ ]:
olsRegressionAnalysis(urban_df,"urban_df","Age-Adjusted Mortality","Household Income")

In [ ]:
olsRegressionAnalysis(rural_df,"rural_df","Age-Adjusted Mortality","Household Income")

In [ ]:
makePanelScatterplot(all_df,"Household Income","Age-Adjusted Mortality","URCS","URCS Name")

# Age adjusted Mortality (Deaths/100k) for Blacks

In [ ]:
makePanelScatterplot(all_df,"Household income (Black)","Age-Adjusted Mortality (Black)","URCS","URCS")

In [ ]:
makePanelScatterplot(all_df,"Household income (Black)","Age-Adjusted Mortality (Black)","URCS","URCS Name")

# Age adjusted Mortality (Deaths/100k) for Whites

In [ ]:
makePanelScatterplot(all_df,"Age-Adjusted Mortality (White)","Household income (White)","URCS","URCS")

In [ ]:
data_df = multi_df
x_data = "Age-Adjusted Mortality (White)"
y_data = "Household income (White)"
color_col = "URCS"
split_col = "URCS Name"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05

g = sns.lmplot(x=x_data, y=y_data, data=all_df,\
           hue=color_col, col=split_col,col_wrap=3,height=4)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max))
      .fig.subplots_adjust(wspace=.02))

# Age adjusted Mortality (Deaths/100k) for Hispanics

In [ ]:
data_df = all_df
x_data = "Household income (Hispanic)"
y_data = "Age-Adjusted Mortality (Hispanic)"
color_col = "URCS"
split_col = "URCS"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
print(f'ylim min:{ylim_min}, ylim max:{ylim_max}')
g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
data_df = all_df
x_data = "Household income (Hispanic)"
y_data = "Age-Adjusted Mortality (Hispanic)"
color_col = "URCS"
split_col = "URCS Name"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
xlim_min = data_df[x_data].min()*1.05
xlim_max = data_df[x_data].max()*1.05

g = sns.lmplot(x=x_data, y=y_data, data=all_df,\
           hue=color_col, col=split_col,col_wrap=3,height=4)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max),xlim=(xlim_min, xlim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
lcm = urban_df.loc[['Large central metro'],x_data].dropna()
lcm.describe()


In [ ]:
sns.distplot(lcm);

In [ ]:
lcm
sns.boxplot(lcm)
# flierprops = dict(markerfacecolor='0.75', markersize=5,
#               linestyle='none')
# seaborn.boxplot(x="centrality", y="score", hue="model", data=data,
#                 flierprops=flierprops)

In [ ]:
sns.jointplot(data=multi_df, x=x_data, y=y_data, kind='reg', color='g')

In [ ]:
multi_df.columns.values

In [ ]:
# View the columns that contain 'Black'
regex4 = '.*Black.*'
cols_black = [col for col in multi_df.columns if re.match(regex4,col) is not None] 
cols_black

In [ ]:
indexes = [0,8,10,11]
for index in sorted(indexes, reverse=True):
    del cols_black[index]
cols_black

In [ ]:
black_df = multi_df[cols_black]
black_df.columns

In [ ]:
black_df.rename(columns={"% LBW (Black)" : "Low_Birth_Weight",
                   "Teen Birth Rate (Black)" : "Teen_Birth_Rate",
                   "Preventable Hosp. Rate (Black)"  : "Prev_Hosp_Rate",
                   "% Screened (Black)"  : "Mammography_Screen_Perct",
                   "% Vaccinated (Black)"  : "Vaccinated_Perct",
                   "% Children in Poverty (Black)"  : "Child_Poverty_Perct",
                   "% Drive Alone (Black)"  : "Drive_Alone_Perct",
                    "Age-Adjusted Mortality (Black)"  : "Age_Adj_Mortality_Rate",
                   "Household income (Black)"  : "Household_Income"     
                   }, inplace=True)

In [ ]:
black_df.head()

In [ ]:
black_urban = black_df.loc['Urban']
black_rural = black_df.loc['Rural']
black_suburbs = black_urban.loc['Large fringe metro']
black_suburbs_clean = black_suburbs.dropna()
black_rural

In [ ]:
black_model = ols("Age_Adj_Mortality_Rate ~ Household_Income + Prev_Hosp_Rate + Child_Poverty_Perct + Vaccinated_Perct", data=black_rural).fit()
print(black_model.summary())

In [ ]:

fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(black_model, fig=fig)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.plot_partregress("Age_Adj_Mortality_Rate", "Household_Income", ["Teen_Birth_Rate", "Prev_Hosp_Rate", "Child_Poverty_Perct","Vaccinated_Perct"],  ax=ax, data=black_suburbs_clean)


In [ ]:
# Teen Birth Rate (Deaths/100k) for All

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate"
y_data = "Age-Adjusted Mortality"
color_col = "URCS"
split_col = "URCS"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
print(f'ylim min:{ylim_min}, ylim max:{ylim_max}')
g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate"
y_data = "Age-Adjusted Mortality"
color_col = "URCS"
split_col = "URCS Name"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
xlim_min = data_df[x_data].min()*1.05
xlim_max = data_df[x_data].max()*1.05

g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col,col_wrap=3,height=4)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max),xlim=(xlim_min, xlim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
# Teen Birth Rate (Deaths/100k) for Black

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate (Black)"
y_data = "Age-Adjusted Mortality (Black)"
color_col = "URCS"
split_col = "URCS"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
print(f'ylim min:{ylim_min}, ylim max:{ylim_max}')
g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate (Black)"
y_data = "Age-Adjusted Mortality (Black)"
color_col = "URCS"
split_col = "URCS Name"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
xlim_min = data_df[x_data].min()*1.05
xlim_max = data_df[x_data].max()*1.05

g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col,col_wrap=3,height=4)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max),xlim=(xlim_min, xlim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
# Teen Birth Rate (Deaths/100k) for White

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate (White)"
y_data = "Age-Adjusted Mortality (White)"
color_col = "URCS"
split_col = "URCS"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
print(f'ylim min:{ylim_min}, ylim max:{ylim_max}')
g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate (White)"
y_data = "Age-Adjusted Mortality (White)"
color_col = "URCS"
split_col = "URCS Name"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
xlim_min = data_df[x_data].min()*1.05
xlim_max = data_df[x_data].max()*1.05

g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col,col_wrap=3,height=4)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max),xlim=(xlim_min, xlim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
# Teen Birth Rate (Deaths/100k) for Hispanic

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate (Hispanic)"
y_data = "Age-Adjusted Mortality (Hispanic)"
color_col = "URCS"
split_col = "URCS"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
print(f'ylim min:{ylim_min}, ylim max:{ylim_max}')
g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
data_df = all_df
x_data = "Teen Birth Rate (Hispanic)"
y_data = "Age-Adjusted Mortality (Hispanic)"
color_col = "URCS"
split_col = "URCS Name"
x_label = x_data
y_label = y_data
ylim_min = data_df[y_data].min()*1.05
ylim_max = data_df[y_data].max()*1.05
xlim_min = data_df[x_data].min()*1.05
xlim_max = data_df[x_data].max()*1.05

g = sns.lmplot(x=x_data, y=y_data, data=data_df,\
           hue=color_col, col=split_col,col_wrap=3,height=4)

g = (g.set_axis_labels(x_label, y_label)
      .set(ylim=(ylim_min, ylim_max),xlim=(xlim_min, xlim_max))
      .fig.subplots_adjust(wspace=.02))

In [ ]:
olsRegressionAnalysis(urban_df,"urban_df","Age-Adjusted Mortality","Teen Birth Rate")

In [ ]:
olsRegressionAnalysis(rural_df,"rural_df","Age-Adjusted Mortality","Teen Birth Rate")

In [ ]:
makePanelScatterplot(all_df,"Age-Adjusted Mortality","Teen Birth Rate (White)","URCS","URCS")

In [ ]:
makePanelScatterplot(all_df,"Age-Adjusted Mortality","Teen Birth Rate (White)","URCS","URCS Name")

In [ ]:
makePanelScatterplot(all_df,"Age-Adjusted Mortality","Teen Birth Rate (Black)","URCS","URCS Name")

In [ ]:
#sort_all_df = all_df.sort_values('Age-Adjusted Mortality',ascending = False)
top_df = all_df.nlargest(5, ['Age-Adjusted Mortality'])
top_1df = top_df[["URCS","URCS Name", "County", "Age-Adjusted Mortality"]]
print('The top five counties: ')
print (top_1df)

In [ ]:
bottom_df = all_df.nlargest(5, ['Age-Adjusted Mortality'])
bottom_1df = bottom_df[["URCS","URCS Name", "County", "Age-Adjusted Mortality"]]
print('The bottom five counties: ')
print (bottom_1df)